# Collating Synthetic Data Results

This notebook is to collate synthetic data results for causal GAN

In [3]:
#imports
import pandas as pd
import collections
import os
import numpy as np
import sys
import plotly.graph_objects as go
import pandas as pd
import copy
debug=False
sys.path.append('generative_models')

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')
master_spec=pd.read_excel('/media/krillman/240GB_DATA/codes2/SSL_GCM/combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier
# all datasets
all_dn=[str(i) for i in master_spec['dataset_spec'].index]
gaussian_results_dict=collections.OrderedDict()

#set idx of gaussian dataset here
ds_idx=[d for d in range(20,21)]

candidate_names=['n{0}_gaussian'.format(k) for k in ds_idx]
rel_dn=np.array([n for n in all_dn if np.any([c in n for c in candidate_names])])
#os.chdir('../jp/')

#imports

#os.chdir('py')
#from results_collate import *


os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/')


#using this one to make function silent
# from https://codereview.stackexchange.com/questions/25417/is-there-a-better-way-to-make-a-function-silent-on-need
class NoStdStreams(object):
    def __init__(self,stdout = None, stderr = None):
        self.devnull = open(os.devnull,'w')
        self._stdout = stdout or self.devnull or sys.stdout
        self._stderr = stderr or self.devnull or sys.stderr

    def __enter__(self):
        self.old_stdout, self.old_stderr = sys.stdout, sys.stderr
        self.old_stdout.flush(); self.old_stderr.flush()
        sys.stdout, sys.stderr = self._stdout, self._stderr

    def __exit__(self, exc_type, exc_value, traceback):
        self._stdout.flush(); self._stderr.flush()
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr
        self.devnull.close()

        
        
def get_acc_ulab(save_folder ,s_i ,model_type):
    if debug:
        set_trace()
    current_dn = '{0}/saved_models/{1}-s_i={2}_unlabel_acc.out'.format(save_folder,model_type,s_i)  # change to this 06_03_2022 ie, need '-' in front of *.ckpt
    acc=np.loadtxt(current_dn)
    return(acc)

def get_acc_test(save_folder ,s_i ,model_type):
    if debug:
        set_trace()
    current_dn = '{0}/saved_models/{1}-s_i={2}_test_acc.out'.format(save_folder,model_type,s_i)    # change to this 06_03_2022 ie, need '-' in front of *.ckpt
    acc=np.loadtxt(current_dn)
    return(acc)


def make_clickable_float(val):
    accuracy='{:.5f}'.format(float(val.split('|')[0]))
    link=val.split('|')[1]
    return f'<a target="_blank" href="{link}">{accuracy}</a>'


def make_float(val):
    accuracy='{:.5f}'.format(float(val))
    return(accuracy)

def make_clickable(val):
    return f'<a target="_blank" href="{val}">plot</a>'



def get_soft_db_path(model_name,d_n,s_i,sdir,hardsoft='soft'):
    im_fn=f'{model_name}_dn_{d_n}_si_{s_i}_{hardsoft}_db_X1.png'
    fpath=f'../{sdir}/saved_models/{im_fn}'
    return(fpath)

import plotly.express as px



model_names = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'CGAN_GUMBEL_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
               'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']





#model_names.sub('CGAN_BASIC_SUPERVISED_CLASSIFIER','CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER')

#need to sub replaceement names
old='CGAN_BASIC_SUPERVISED_CLASSIFIER'
new='CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER'

mn=[new if m==old else m for m in model_names]

old='CGAN_GUMBEL_SUPERVISED_CLASSIFIER'
new='CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER'
mn=[new if m==old else m for m in mn]
mn

model_names_synthetic=mn 

In [4]:
model_names = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'CGAN_GUMBEL_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
               'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']



n_models=len(model_names)

px_colours=px.colors.qualitative.Plotly
px_colours=px_colours[:n_models]
#len(px_colours)

color_discrete_sequence=["red","lightgreen","blue","orange","grey","magenta","brown","purple","darkgreen","chocolate"]

colours_dict={k:v for k,v in zip(model_names,px_colours)}


#read the colour mapping
colour_map_fn='/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/model_colours.tex'
with open(colour_map_fn,'r') as f:    
    all_colours=f.readlines()
#now do some subsition
col_list=[ac.replace('\\definecolor{','').replace('}{HTML}{',' ').replace('}\n','') for ac in all_colours]
#convert him to a nice dict|
col_dict={l.split(' ')[0]:'#'+l.split(' ')[1] for l in col_list}
#overwrite prev
colours_dict=col_dict


import json

with open('colours_dict.json', 'w') as fp:
    json.dump(colours_dict, fp)
    

In [ ]:
col_dict

Ok this need to be imported so that we can write example of colour to the table

In [ ]:
# quadratic kernel will map following elements:

# a: x2x1
# b: x2**2
# c: x1**2
# d: x1
# e: x2

import pandas as pd
import numpy as np
import plotly.express as px

#get terms for polynomial kernel of degree=2
def get_polynomial_d2_terms(in_df):
    retval={}
    retval['X1^2']=in_df.X1**2
    retval['X2^2']=in_df.X2**2
    retval['X1']=in_df.X1
    retval['X2']=in_df.X2
    retval['X1X2']=in_df.X1*in_df.X2
    return(retval)

def return_d1():
    #now, let's say these rows each are element of polynomial kernel, of degree 2
    #run x1,x2 thru each and plot if greater/lesser than, as per what is written directly above ^^

    condition='not_met'
    
    while condition=='not_met':
        #generate the hyperparameters for our polynomial kernel
        rnd_init=np.random.random(size=(100, 5))
        kernel_params=pd.DataFrame(rnd_init,columns=['X1^2','X2^2','X1','X2','X1X2'])

        #generate some random x
        mean = (1, 1)
        cov = [[1, 0], [0, 1]]
        n_x_examples=1000
        x1x2=pd.DataFrame(np.random.multivariate_normal(mean, cov, (n_x_examples)),columns=['X1','X2'])


        #find polynomial kernel terms for our data
        pt=get_polynomial_d2_terms(x1x2)
        pt=pd.DataFrame(pt) #convert to dataframe

        #index of row for testing our polynomial kernel
        testrow=0 
        #df.loc[testrow,]

        test_k=pt*kernel_params.loc[testrow,] #multiply by kernel elements
        test_k['sum']=test_k.sum(axis=1) #sum results. note this is equivalent to W^(T)*X
        test_k['class']=(test_k['sum']>1).astype(int)


        tm=0.5 # if mean==0.5, classes are balanced exactly

        dm=0.1 #acceptable deviation in the mean ie we need mean +- deviation_mean/2

        mean_lower=tm-dm/2
        mean_upper=tm+dm/2

        mu_s=test_k['class'].mean() #sample mu

        if mean_lower<mu_s and mu_s<=mean_upper:
            condition='met'
            #accept this one
            #which means that we return it
            ret_dict={}
            ret_dict={'kernel_params':kernel_params,
                     'kernel_calc':test_k,
                     'sample_mu':mu_s,
                     'x1x2y':pd.concat([x1x2,test_k['class']],axis=1)}
    
    return(ret_dict)


# and we need to map like this:
# W=[a,b,c,d,e]
# X=[x2x1,x2**2,x1**2,x1,x2]
# W^(T)X = C
# ie a matrix multiplication that results in some constant C
# and we can take C as the decision boundary, so that W^(T)X > C ==1, and 0 otherwise

n_dset=1

d1_sets=[return_d1() for k in range(n_dset)]


df=d1_sets[0]['x1x2y']
px.scatter(df,x='X1',y='X2',color='class',color_continuous_scale='bluered_r',opacity=0.3)

#### Synchronising remote slurm output to local

```SH

!rm -rf /Users/macuser/Documents/GitHub/causal_ssl_gan/slurm/slurm_results/ ; rsync -abviuzPI apmoore@spartan.hpc.unimelb.edu.au:/data/projects/punim1573/amoore/causal_ssl_gan/slurm/slurm_results/ /Users/macuser/Documents/GitHub/causal_ssl_gan/slurm/slurm_results/ ;```

# Writing some code to compute MMD between synthetic and true dist



In [ ]:
import os
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')
import sys
sys.path.append('generative_models')
sys.path.append('py')
sys.path.append('py/generative_models/')

import copy
import argparse
from collections import OrderedDict
from generative_models.Generator_Y_from_X1 import *
from generative_models.Generator_X2_from_Y import *
from generative_models.Generator_X2_from_YX1 import *
from generative_models.Generator_Y import *
from generative_models.Generator_X1 import *
from generative_models.Generator_X_from_X import *
from gen_data_loaders import *
from generative_models.benchmarks_cgan import *
#from parse_data import *
import time

os.chdir('jp')

# load dsc.merge_dat



def get_mmd_dist(sdir,d_n,s_i,dspec):
    
    #os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')
    #os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')
    dsc_loader=eval(dspec.loc[d_n].dataloader_function) #within the spec



    args=Args(d_n)

    dsc=dsc_loader(args,s_i,dspec.loc[d_n])
    dsc=manipulate_dsc(dsc,dspec.loc[d_n]) #adding extra label column and convenient features for complex data mod later on

    #os.chdir('jp')
    feature_cols=[c for c in dsc.merge_dat.columns if c[0]=='X']


    unlabelled_dsc=dsc.merge_dat[dsc.merge_dat.type=='unlabelled']
    
    #unlabelled_dsc.sort_values(by='Y',inplace=True)

    synthetic_data_fn=f'{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}.csv'
    synthetic_data=pd.read_csv(synthetic_data_fn,index_col=0)


    min_rows=min(unlabelled_dsc.shape[0],synthetic_data.shape[0])



    unlabelled_dsc=unlabelled_dsc.sample(min_rows,replace=False)
    synthetic_data=synthetic_data.sample(min_rows,replace=False)
    
    
    
    
    

    mpwd=get_median_pwd(torch.tensor(unlabelled_dsc[feature_cols].values))

    sigma_list=[mpwd * k for k in [1/4,1/2,1,2,4]]

    #get RBF of synthetic data...


    #rename Y_0 to Y

    scols=[c.replace('Y_0','Y') for c in synthetic_data.columns]
    scols
    synthetic_data.columns=scols

    
    unlabelled_dsc.sort_values(by='Y',inplace=True)
    synthetic_data.sort_values(by='Y',inplace=True)
    
    sd_dict={}
    sd_dict['features']=torch.tensor(synthetic_data[feature_cols].values)
    sd_dict['label']=torch.tensor(torch.nn.functional.one_hot(torch.tensor(synthetic_data['Y'].values),2))


    od_dict={}
    od_dict['features']=torch.tensor(unlabelled_dsc[feature_cols].values)
    od_dict['label']=torch.tensor(torch.nn.functional.one_hot(torch.tensor(unlabelled_dsc['Y'].values),2))

    mmd_dist=mix_rbf_mmd2_joint(sd_dict['features'],od_dict['features'],sd_dict['label'],od_dict['label'],sigma_list=sigma_list)
    return(mmd_dist.cpu().detach().item())

#mmd_dist



In [ ]:
# ok just cheat and combine all the 'X' into joint MMD - will only work for synthetic data

NameError: name 'model_names' is not defined

In [ ]:
model_names_synthetic

### d1: n36

In [4]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d1'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['mmd_diff']=list_of_mmd
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on test data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    
    
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')      
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,data_plot,synthetic_cgan_data,mmd_diff
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
5,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
6,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
7,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0
9,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,plot,plot,0


#### Violin Plots

In [ ]:
#box plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d1 = ['FULLY_SUPERVISED_CLASSIFIER',
                  'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
               'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d1]

for m in model_names_d1:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]

fig = go.Figure()
for m in model_names_d1:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#        source='causal_graphs/cg1.png',
#        x=0.7,
#        y=1,
#        )
#)


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/test_acc_g1.png")



### Accuracy (unlabelled)

In [ ]:

#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d1'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['mmd_diff']=list_of_mmd
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on ulab data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    
    
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')      
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#box plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d1 = ['FULLY_SUPERVISED_CLASSIFIER',
                  'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
               'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d1]

for m in model_names_d1:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]

fig = go.Figure()
for m in model_names_d1:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#        source='causal_graphs/cg1.png',
#        x=0.7,
#        y=1,
#        )
#)


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/ulab_acc_g1.png",scale=5)



### d2: n36

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d2'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
     
    ma['mmd_diff']=list_of_mmd
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    
    #ma['data_plot']=list_of_plots
    #ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on test data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')      
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)
#ma

#### Violin Plots

In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)
ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})



model_names_d2 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d2]

for m in model_names_d2:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]
    
    
#get colours



fig = go.Figure()
for m in model_names_d2:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#fig.update_yaxes(range=list(...))
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/test_acc_g2.png")


### Accuracy (unlabelled)

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d2'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
     
    ma['mmd_diff']=list_of_mmd
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    
    #ma['data_plot']=list_of_plots
    #ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on ulab data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')      
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)
#ma

#violin plots for model performamces
ma_i=copy.deepcopy(ma)
ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})



model_names_d2 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d2]

for m in model_names_d2:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]
    
    
#get colours



fig = go.Figure()
for m in model_names_d2:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#fig.update_yaxes(range=list(...))
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/ulab_acc_g2.png",scale=5)


### d3: n36

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d3'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


#### Violin Plots

In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})

model_names_d3 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                'CGAN_GUMBEL_SUPERVISED_CLASSIFIER',  
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)



#fig.update_layout(legend=dict(font=dict(size= 20)))#

#fig.update_layout(legend= {'itemsizing': 'constant'})#,marker={'size':100}))

#fig.update_layout(
#    width=5000,
#    height=2500)

fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/test_acc_g3.png")

#fig = go.Figure(data=data, layout=layout)

fig.show()


### Accuracy (unlabelled)

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d3'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)

#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})

model_names_d3 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                'CGAN_GUMBEL_SUPERVISED_CLASSIFIER',  
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)



#fig.update_layout(legend=dict(font=dict(size= 20)))#

#fig.update_layout(legend= {'itemsizing': 'constant'})#,marker={'size':100}))

#fig.update_layout(
#    width=5000,
#    height=2500)

fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/ulab_acc_g3.png",scale=5)

#fig = go.Figure(data=data, layout=layout)

fig.show()



### d4: n36

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d4'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si


with NoStdStreams():
    for current_model in model_names:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names }
    

    style_dict={}
    style_dict={m:make_float for m in model_names}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

os.chdir('jp')
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


#### Violin Plots

In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d4 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']



ma_i=ma_i[model_names_d4]
ma_i=ma_i.astype(float)


fig = go.Figure()
for m in model_names_d4:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/test_acc_g4.png")



### Accuracy (unlabelled)

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d4'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si


with NoStdStreams():
    for current_model in model_names:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names }
    

    style_dict={}
    style_dict={m:make_float for m in model_names}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

os.chdir('jp')
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d4 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']



ma_i=ma_i[model_names_d4]
ma_i=ma_i.astype(float)


fig = go.Figure()
for m in model_names_d4:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/ulab_acc_g4.png",scale=5)




### d5: n36

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d5'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


#### Violin Plots

In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/test_acc_g5.png")





### Accuracy (unlabelled)

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d5'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)

#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/ulab_acc_g5.png",scale=5)








### d6: n36

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d6'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



#model_names_synthetic.insert(3,'CGAN_GUMBEL_DJ_XCES_SUPERVISED_CLASSIFIER')


with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


#### Violin Plots

In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})



ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/test_acc_g6.png")





### Accuracy (unlabelled)

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d6'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



#model_names_synthetic.insert(3,'CGAN_GUMBEL_DJ_XCES_SUPERVISED_CLASSIFIER')


with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})



ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/ulab_acc_g6.png",scale=5)






### d7: n36

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d7'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


#### Violin Plots

In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/test_acc_g7.png")




### Accuracy (unlabelled)

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='n36_gaussian_mixture_d7'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/collating_results/images/ulab_acc_g7.png",scale=5)





# Collating Real Data Results

## Sachs: RAF

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='real_sachs_raf_log'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names }
    

    style_dict={}
    style_dict={m:make_float for m in model_names}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)
ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("../causal_ssl_gan_paper/images/real_boxplot/ulab_acc_sachs_raf.png",scale=5)


## Sachs: MEK

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='real_sachs_mek_log'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names }
    

    style_dict={}
    style_dict={m:make_float for m in model_names}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)
ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("../causal_ssl_gan_paper/images/real_boxplot/ulab_acc_sachs_mek.png",scale=5)




## Bcancer Wisconsin: ZScore

In [ ]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

d_n='real_bcancer_diagnosis_zscore'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names }
    

    style_dict={}
    style_dict={m:make_float for m in model_names}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


In [ ]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)
ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

fig.update_layout(yaxis_range=[0.65,1])

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("../causal_ssl_gan_paper/images/real_boxplot/ulab_acc_bcancer_diagnosis_zscore.png",scale=5)




# Example output for illustrating Van Kleugen et al (2019) conditional cluster assumption

In [ ]:
#from sklearn import datasets

In [ ]:
import sklearn
import numpy as np
import scipy
import pandas as pd
import copy
import plotly.express as px
import plotly.graph_objects as go


#c1 = np.random.normal(0, 0.5, 1000)


def return_norm_samples(class_label,n=100):
    if class_label==0:
        return(np.random.normal(-2, 0.5, n))
    if class_label==1:
        return(np.random.normal(2, 0.5, n))



#get our Y class label:

vank_example=pd.DataFrame([0]*1000 + [1]*1000)
vank_example.columns=['Y']
vank_example['X_C']=[return_norm_samples(y,1)[0] for y in vank_example['Y']]
vank_example['X_E']=[return_norm_samples(np.fmod(y+1,2),1)[0] for y in vank_example['Y']]


vank_example['Y']=vank_example['Y'].astype(str)
#px.histogram(return_norm_samples(1))

#make copy
vo=copy.deepcopy(vank_example)
vc=copy.deepcopy(vank_example)
vc['X_E']=-vc['X_E']
vc['Y']='0'
px.scatter(vc, x=vc.X_C, y=vc.X_E, color=vc.Y)


vank_example['Y']='1'


g_df=pd.concat([vc,vank_example])

#px.scatter(vank_example, x=g_df.X_C, y=g_df.X_E, color=g_df.Y)
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=vc.X_C, y=vc.X_E,
                    mode='markers',
                    name='',marker_color='red',marker_symbol='circle'))
# Add traces
fig.add_trace(go.Scatter(x=vo.X_C, y=vo.X_E,
                    mode='markers',
                    name='',marker_color='blue',marker_symbol='circle'))





fig.add_trace(go.Scatter(x=[-4,4],y=[4, -4],mode="lines",line=go.scatter.Line(color="blue"),showlegend=False))

fig.add_trace(go.Scatter(x=[-4,4],y=[-4, 4],mode="lines",line=go.scatter.Line(color="red"),showlegend=False))


fig.update_layout(showlegend=False)
fig.update_layout(
    paper_bgcolor='rgba(255,255,255,1)',
    plot_bgcolor='rgba(255,255,255,1)',
    xaxis_title=r'$\Huge X_C$',
    yaxis_title=r'$\Huge X_E$')


fig.update_layout(title_text="",
        paper_bgcolor='rgba(255,255,255,1)',\
    plot_bgcolor='rgba(255,255,255,1)',\
                  title_font_size=30)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
#fig.update_xaxes(zeroliene=True),#minor=dict(showgrid=False))
#fig.update_yaxes(minor_ticks="inside")






fig.update_layout(
    # on the y-axis
    yaxis_ticksuffix=r"$",
    yaxis_tickprefix=r"$\Huge ",
    xaxis_ticksuffix=r"$",
    xaxis_tickprefix=r"$\Huge ",
    yaxis_showticksuffix="all"  # or "first" or "last"
)

fig.update_xaxes(ticks="outside")
fig.update_yaxes(ticks="outside")

fig.update_yaxes( title_standoff = 25)



fig.write_image('../causal_ssl_gan_paper/images/vank_ssl_xe_xc.png', scale=5)



fig = go.Figure()
#fig.add_trace(go.Histogram(x=vc.X_E))
#fig.add_trace(go.Histogram(x=vo.X_E))

fig.layout.legend.tracegroupgap = 200


fig.add_trace(go.Histogram(
    histnorm='percent',
    x=vc.X_E,

    name=r'$\Huge Y=1\\\\$', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=-4.0,
        end=4,
        size=0.05
    ),
    marker_color='red',
    opacity=0.5
)         
        
             )


fig.add_trace(go.Histogram(
    histnorm='percent',
    x=vo.X_E,
    name=r'$ \Huge  Y=0\\\\$', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=-4.0,
        end=4,
        size=0.05
    ),
    marker_color='blue',
    opacity=0.5
))


# Overlay both histograms
fig.update_xaxes(rangeselector_font=dict(size=100))

fig.update_layout(barmode='overlay')
fig.update_layout(xaxis_title=r'$\Huge X$')
fig.update_layout(yaxis_title=r'$\Huge P(X)$')

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')


# update_layout method used to modify change and size
#fig.update_layout(legend=dict(font=dict(size= 1)))

fig.update_layout(
    paper_bgcolor='rgba(255,255,255,1)',
    plot_bgcolor='rgba(255,255,255,1)')


fig.update_layout(
    # on the y-axis
    yaxis_ticksuffix=r"$",
    yaxis_tickprefix=r"$\Huge ",
    xaxis_ticksuffix=r"$",
    xaxis_tickprefix=r"$\Huge ",
    yaxis_showticksuffix="all"  # or "first" or "last"
)

fig.update_xaxes(ticks="outside")
fig.update_yaxes(ticks="outside")

fig.update_yaxes( title_standoff = 25)


fig.write_image('../causal_ssl_gan_paper/images/vank_ssl_xe_hist.png', scale=5)


#fig.show()

In [ ]:
fig.show()

# Results table formatting: Pandas -> Latek with auto bold for highest value

13_01_2023

In [ ]:
# https://stackoverflow.com/questions/65070070/highlight-the-best-value-of-each-row-in-python-pandas-to-latex

from functools import partial

import pandas as pd
import numpy as np
import os
os.makedirs('latek_fiddling',exist_ok=True)

def bold_formatter(x, value):
    """Format a number in bold when (almost) identical to a given value.
    
    Args:
        x: Input number.
        
        value: Value to compare x with.
        
        num_decimals: Number of decimals to use for output format.

    Returns:
        String converted output.

    """
    # Consider values equal, when rounded results are equal
    # otherwise, it may look surprising in the table where they seem identical
    if round(x, num_decimals) == round(value, num_decimals):
        return f"\\textbf{{{x}}}"
    else:
        return x


    
drop_idx=['n','psup_baseline']
def get_colmax(in_col,drop_idx=drop_idx):
    #takes in_col as pd series (column) and returns the max value when interpreted as value+-error
    #drop some columns...
    in_values=in_col.drop(drop_idx).values
    values=[v.split(' ± ')[0] for v in in_values]
    values=[float(v) for v in values]
    values=np.array(values)
    max_idx=np.argmax(values)
    max_instance=in_values[max_idx]
    return(max_instance)


fmt_bold_max = {column: partial(bold_formatter, value=get_colmax(df[column])) for column in max_columns_2f}
fmts = dict(**fmt_bold_max)

with open("latek_fiddling/test_table.tex", "w") as fh:
    df.to_latex(buf=fh,
                index=False,
                header=col_names,
                formatters=fmts,
                escape=False)

In [ ]:



value = return_colmax_str(df[column])

# takes pandas  series ie df[column] =and then converts back to float and then returns the max value
# as string including error bars



In [ ]:


def format_w_error(mean,std):
    retval='{:.3f} ± {:.3f}'.format(mean * 100,std* 100)
    return(retval)


    


In [ ]:
import numpy as np

mv=np.random.rand(10)
sv=np.random.rand(10)


vals=[format_w_error(m,s) for m,s in zip(mv,sv)]
max_val=get_colmax(pd.Series(vals))

print(max_val)
vals

# Collate Synthetic retrials for 10000,100000 unlabelled

## 10,000

### d1

In [5]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d1_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['mmd_diff']=list_of_mmd
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on test data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    
    
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')       
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,data_plot,synthetic_cgan_data,mmd_diff
0,0.54100,0.51200,nan,0.51000,0.51500,0.50100,0.50000,0.52200,0.49300,0.50300,plot,plot,0
1,0.74000,0.63000,nan,0.63900,0.62200,0.65100,0.58400,0.60600,0.63800,0.62500,plot,plot,0
2,0.63972,0.63373,nan,0.65669,0.64471,0.63972,0.64371,0.61277,0.64471,0.64970,plot,plot,0
3,0.67900,0.59800,nan,0.60600,0.60200,0.61200,0.60900,0.59800,0.60200,0.61000,plot,plot,0
4,0.70529,0.61838,nan,0.60040,0.64635,0.60340,0.55544,0.64935,0.66733,0.63636,plot,plot,0
5,0.69630,0.62737,nan,0.60340,0.69031,0.65435,0.66533,0.61938,0.60539,0.65634,plot,plot,0
6,0.71400,0.68300,nan,0.65700,0.68600,0.64600,0.64200,0.70100,0.64900,0.70100,plot,plot,0
7,0.67632,0.61938,nan,0.62238,0.64036,0.63636,0.63836,0.61938,0.60440,0.63437,plot,plot,0
8,0.62238,0.54745,nan,0.56144,0.56943,0.55644,0.56044,0.58941,0.56843,0.59141,plot,plot,0
9,0.70600,0.62300,nan,0.63300,0.62800,0.60800,0.59700,0.62400,0.59700,0.61800,plot,plot,0


#### Violin Plots

In [6]:
#box plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d1 = ['FULLY_SUPERVISED_CLASSIFIER',
                  'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
               'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d1]

for m in model_names_d1:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]

fig = go.Figure()
for m in model_names_d1:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#        source='causal_graphs/cg1.png',
#        x=0.7,
#        y=1,
#        )
#)


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g1_10000.png")



### Accuracy (unlabelled)

In [7]:

#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d1_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['mmd_diff']=list_of_mmd
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on ulab data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    
    
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')       
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#box plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d1 = ['FULLY_SUPERVISED_CLASSIFIER',
                  'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
               'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d1]

for m in model_names_d1:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]

fig = go.Figure()
for m in model_names_d1:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#        source='causal_graphs/cg1.png',
#        x=0.7,
#        y=1,
#        )
#)


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g1_10000.png",scale=5)
#fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g1_10000.png")



### d2: n36

In [16]:
#rel_dn

#os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d2_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
     
    ma['mmd_diff']=list_of_mmd
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    
    #ma['data_plot']=list_of_plots
    #ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on test data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
ma.drop(columns=['s_i'],inplace=True)





ma.sort_values(by=list(ma.columns)[1],inplace=True)



ma.style.format(style_dict)
#ma

,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,mmd_diff,data_plot,synthetic_cgan_data
29,0.97800,0.72500,nan,0.91300,0.86600,0.90200,0.88900,0.89600,0.79600,0.89600,0,plot,plot
1,0.94500,0.76100,nan,0.89100,0.87400,0.85200,0.82800,0.86600,0.56400,0.88500,0,plot,plot
82,0.97300,0.76100,nan,0.82500,0.76200,0.80600,0.75900,0.77100,0.89563,0.82300,0,plot,plot
19,0.98400,0.76600,nan,0.91000,0.84400,0.87400,0.84700,0.72700,0.92500,0.72500,0,plot,plot
36,0.95604,0.76623,nan,0.96404,0.85015,0.79820,0.86414,0.83716,0.85315,0.78322,0,plot,plot
33,0.98300,0.77100,nan,0.83800,0.86300,0.86300,0.75400,0.87500,0.83900,0.91300,0,plot,plot
98,0.97802,0.77722,nan,0.92128,0.87762,0.89187,0.89082,0.89071,0.89563,0.92208,0,plot,plot
9,0.99600,0.78000,nan,0.89800,0.88900,0.84200,0.87400,0.87000,0.91300,0.77300,0,plot,plot
58,0.98900,0.79000,nan,0.87900,0.85500,0.91300,0.82400,0.79900,0.86700,0.78200,0,plot,plot
43,0.96004,0.79520,nan,0.92008,0.92008,0.85614,0.85514,0.86414,0.89211,0.85614,0,plot,plot


In [5]:
#rel_dn

#os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d2_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
     
    ma['mmd_diff']=list_of_mmd
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    
    #ma['data_plot']=list_of_plots
    #ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on test data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
ma.drop(columns=['s_i'],inplace=True)





ma.sort_values(by=list(ma.columns)[1],inplace=True)



ma.style.format(style_dict)
#ma

,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,mmd_diff,data_plot,synthetic_cgan_data
19,0.98400,0.76700,nan,0.91000,0.84400,0.87400,0.84700,0.72700,0.92500,0.72500,0,plot,plot
9,0.99600,0.77100,nan,0.89800,0.88900,0.84200,0.87400,0.87000,0.91300,0.77300,0,plot,plot
98,0.97802,0.77722,nan,0.92128,0.87762,0.89187,0.89082,0.89071,0.89563,0.92208,0,plot,plot
82,0.97300,0.78700,nan,0.82500,0.76200,0.80600,0.75900,0.77100,0.89563,0.82300,0,plot,plot
58,0.98900,0.79000,nan,0.87900,0.85500,0.91300,0.82400,0.79900,0.86700,0.78200,0,plot,plot
44,0.88800,0.79700,nan,0.89000,0.87800,0.80700,0.87000,0.82700,0.89900,0.81300,0,plot,plot
33,0.98300,0.80800,nan,0.83800,0.86300,0.86300,0.75400,0.87500,0.83900,0.91300,0,plot,plot
96,0.96500,0.80900,nan,0.92128,0.87762,0.89187,0.89082,0.89071,0.89563,0.82600,0,plot,plot
64,0.95300,0.81200,nan,0.90400,0.81000,0.87300,0.89900,0.85900,0.85700,0.86000,0,plot,plot
65,0.96400,0.81300,nan,0.87700,0.82600,0.86000,0.89100,0.88300,0.78500,0.88300,0,plot,plot


#### Violin Plots

In [9]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)
ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})



model_names_d2 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d2]

for m in model_names_d2:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]
    
    
#get colours



fig = go.Figure()
for m in model_names_d2:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#fig.update_yaxes(range=list(...))
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g2_10000.png")


### Accuracy (unlabelled)

In [1]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d2_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_mmd=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        #list_of_mmd.append(get_mmd_dist(sdir,d_n,s_i,dspec))
        list_of_mmd.append(0)
     
    ma['mmd_diff']=list_of_mmd
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    
    #ma['data_plot']=list_of_plots
    #ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    
    #split into performance on ulab data, and path to the decision boundary plot, separated by '|' as delimiter
    for m in model_names_synthetic:
        ma[m]=ma[m]+'|'+dict_of_paths[m]
        
    style_dict={m:make_clickable_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

    
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')      
#os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)
#ma

#violin plots for model performamces
ma_i=copy.deepcopy(ma)
ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})



model_names_d2 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names_d2]

for m in model_names_d2:
    ma_i[m]=[float(result.split('|')[0]) for result in ma_i[m]]
    
    
#get colours



fig = go.Figure()
for m in model_names_d2:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#fig.update_yaxes(range=list(...))
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
#fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g2_10000.png",scale=5)
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g2_10000.png")


NameError: name 'os' is not defined

### d3: n36

In [13]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d3_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)



ma.sort_values(by='CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER',inplace=True,ascending=False)



ma.style.format(style_dict)


,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,data_plot,synthetic_cgan_data,synthetic_gumbel_cgan_data
35,0.99100,0.86600,nan,0.89600,0.80000,0.85800,0.89900,0.82800,0.86500,0.78100,plot,plot,plot
70,0.97900,0.86200,nan,0.91800,0.70700,0.82700,0.81900,0.79800,0.83600,0.74500,plot,plot,plot
18,0.99300,0.85600,nan,0.86100,0.72200,0.83900,0.84100,0.87600,0.86600,0.84400,plot,plot,plot
65,0.97600,0.85200,nan,0.87300,0.78200,0.86900,0.81200,0.71200,0.88000,0.76400,plot,plot,plot
8,0.98402,0.84615,nan,0.81818,0.77223,0.84216,0.84815,0.75325,0.80619,0.79121,plot,plot,plot
28,0.97500,0.84400,nan,0.84600,0.66400,0.76500,0.73500,0.81000,0.70600,0.78200,plot,plot,plot
92,0.99100,0.84300,nan,0.81461,0.69142,0.75266,0.73920,0.74442,0.72634,0.78900,plot,plot,plot
56,0.98500,0.84300,nan,0.89500,0.62300,0.92800,0.78100,0.86000,0.88500,0.86000,plot,plot,plot
38,0.96600,0.82600,nan,0.87400,0.77100,0.83900,0.89100,0.88900,0.88700,0.88300,plot,plot,plot
84,0.98700,0.82400,nan,0.81461,0.69142,0.75266,0.73920,0.74442,0.72634,0.84500,plot,plot,plot


#### Violin Plots

In [25]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})

model_names_d3 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                'CGAN_GUMBEL_SUPERVISED_CLASSIFIER',  
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)



#fig.update_layout(legend=dict(font=dict(size= 20)))#

#fig.update_layout(legend= {'itemsizing': 'constant'})#,marker={'size':100}))

#fig.update_layout(
#    width=5000,
#    height=2500)

fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g3_10000.png")

#fig = go.Figure(data=data, layout=layout)

fig.show()


### Accuracy (unlabelled)

In [27]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d3_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)

#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})

model_names_d3 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                'CGAN_GUMBEL_SUPERVISED_CLASSIFIER',  
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)



#fig.update_layout(legend=dict(font=dict(size= 20)))#

#fig.update_layout(legend= {'itemsizing': 'constant'})#,marker={'size':100}))

#fig.update_layout(
#    width=5000,
#    height=2500)

fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g3_10000.png",scale=5)

#fig = go.Figure(data=data, layout=layout)

fig.show()



### d4: n36

In [29]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d4_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si


with NoStdStreams():
    for current_model in model_names:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names }
    

    style_dict={}
    style_dict={m:make_float for m in model_names}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})



os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  


    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,data_plot,synthetic_cgan_data
0,0.99700,nan,nan,0.80619,0.68232,0.75025,0.60040,0.74426,0.75225,0.71029,plot,plot
1,0.99100,nan,nan,0.89500,0.74000,0.91600,0.87200,0.82700,0.86800,0.83000,plot,plot
2,0.99000,nan,nan,0.68100,0.57900,0.56900,0.69200,0.59400,0.59900,0.67200,plot,plot
3,0.97600,nan,nan,0.80700,0.76300,0.82500,0.79500,0.85800,0.87600,0.86600,plot,plot
4,0.95400,nan,nan,0.89000,0.86700,0.89900,0.89500,0.81400,0.92400,0.88700,plot,plot
5,0.96100,nan,nan,0.91200,0.73200,0.81900,0.76400,0.78900,0.92900,0.80500,plot,plot
6,0.97802,nan,nan,0.83117,0.75025,0.75724,0.83217,0.79421,0.83417,0.85814,plot,plot
7,0.99400,nan,nan,0.72800,0.65400,0.73800,0.64100,0.72600,0.72100,0.70900,plot,plot
8,0.95800,nan,nan,0.90100,0.72900,0.82500,0.83800,0.84500,0.90500,0.84600,plot,plot
9,0.99600,nan,nan,0.91200,0.68900,0.84900,0.83400,0.83700,0.96000,0.83700,plot,plot


#### Violin Plots

In [31]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d4 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']



ma_i=ma_i[model_names_d4]
ma_i=ma_i.astype(float)


fig = go.Figure()
for m in model_names_d4:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g4_10000.png")



### Accuracy (unlabelled)

In [33]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d4_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si


with NoStdStreams():
    for current_model in model_names:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names }
    

    style_dict={}
    style_dict={m:make_float for m in model_names}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable})

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  


    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER'})


model_names_d4 = ['FULLY_SUPERVISED_CLASSIFIER',
               'CGAN_BASIC_SUPERVISED_CLASSIFIER',
               'VAT',
               'TRIPLE_GAN',
               'SSL_GAN',
               'SSL_VAE',
               'ENTROPY_MINIMISATION',
                'LABEL_PROPAGATION',
               'PARTIAL_SUPERVISED_CLASSIFIER']



ma_i=ma_i[model_names_d4]
ma_i=ma_i.astype(float)


fig = go.Figure()
for m in model_names_d4:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))

#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)
#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g4_10000.png",scale=5)




### d5: n36

In [34]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM')

d_n='n36_gaussian_mixture_d5_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,data_plot,synthetic_cgan_data,synthetic_gumbel_cgan_data
0,0.98100,nan,nan,0.86000,0.72400,0.83400,0.78200,0.78200,0.66600,0.82900,plot,plot,plot
1,0.99401,nan,nan,0.83716,0.83716,0.79121,0.81019,0.83716,0.85614,0.84016,plot,plot,plot
2,0.97700,nan,nan,0.85000,0.72100,0.71000,0.61600,0.64300,0.62800,0.64300,plot,plot,plot
3,0.99300,nan,nan,0.91700,0.80600,0.79200,0.84200,0.78000,0.87500,0.79300,plot,plot,plot
4,0.98002,nan,nan,0.85015,0.64635,0.77822,0.72228,0.80320,0.68332,0.75624,plot,plot,plot
5,0.98600,nan,nan,0.92700,0.89600,0.83400,0.89600,0.89600,0.94700,0.89200,plot,plot,plot
6,0.99300,nan,nan,0.83100,0.72000,0.68300,0.78800,0.72500,0.68300,0.77600,plot,plot,plot
7,0.98600,nan,nan,0.94500,0.79200,0.81700,0.74500,0.86200,0.76300,0.82700,plot,plot,plot
8,0.99300,nan,nan,0.88600,0.86000,0.64600,0.77000,0.81900,0.89300,0.75100,plot,plot,plot
9,0.98200,nan,nan,0.87300,0.72200,0.78800,0.77400,0.82200,0.83200,0.85000,plot,plot,plot


#### Violin Plots

In [36]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g5_10000.png")





### Accuracy (unlabelled)

In [38]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM')

d_n='n36_gaussian_mixture_d5_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)

#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g5_10000.png",scale=5)








### d6: n36

In [39]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d6_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



#model_names_synthetic.insert(3,'CGAN_GUMBEL_DJ_XCES_SUPERVISED_CLASSIFIER')


with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,data_plot,synthetic_cgan_data,synthetic_gumbel_cgan_data
0,0.98600,nan,nan,0.82700,0.71700,0.75500,0.67600,0.76600,0.76400,0.76500,plot,plot,plot
1,0.99800,nan,nan,0.79620,0.74426,0.71229,0.70629,0.64436,0.70929,0.75025,plot,plot,plot
2,0.98701,nan,nan,0.82717,0.75025,0.77423,0.76723,0.77822,0.78122,0.73427,plot,plot,plot
3,0.98600,nan,nan,0.88000,0.79200,0.83000,0.74000,0.74900,0.79100,0.78500,plot,plot,plot
4,0.97900,nan,nan,0.88100,0.64300,0.70900,0.68500,0.70800,0.74800,0.70600,plot,plot,plot
5,0.99800,nan,nan,0.87500,0.71800,0.81000,0.80900,0.77600,0.80500,0.80100,plot,plot,plot
6,0.99400,nan,nan,0.76500,0.78100,0.74300,0.75500,0.77300,0.73200,0.74100,plot,plot,plot
7,0.97200,nan,nan,0.77900,0.73900,0.80400,0.69500,0.75200,0.78600,0.65500,plot,plot,plot
8,0.96000,nan,nan,0.72400,0.68000,0.67500,0.66200,0.69900,0.65800,0.66300,plot,plot,plot
9,0.97000,nan,nan,0.76500,0.76800,0.78300,0.82300,0.81900,0.79600,0.77800,plot,plot,plot


#### Violin Plots

In [40]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})



ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g6_10000.png")





### Accuracy (unlabelled)

In [41]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d6_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si



#model_names_synthetic.insert(3,'CGAN_GUMBEL_DJ_XCES_SUPERVISED_CLASSIFIER')


with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})



ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g6_10000.png",scale=5)






### d7: n36

In [42]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d7_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_test(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
ma.drop(columns=['s_i'],inplace=True)
ma.style.format(style_dict)


,FULLY_SUPERVISED_CLASSIFIER,CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER,CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER,VAT,TRIPLE_GAN,SSL_GAN,SSL_VAE,ENTROPY_MINIMISATION,LABEL_PROPAGATION,PARTIAL_SUPERVISED_CLASSIFIER,data_plot,synthetic_cgan_data,synthetic_gumbel_cgan_data
0,0.99000,nan,nan,0.74100,0.63700,0.73500,0.65100,0.64400,0.67100,0.67100,plot,plot,plot
1,0.98202,nan,nan,0.84416,0.63636,0.66234,0.64336,0.67433,0.70829,0.69131,plot,plot,plot
2,0.98700,nan,nan,0.86000,0.82000,0.82900,0.69100,0.76800,0.78500,0.76400,plot,plot,plot
3,0.96600,nan,nan,0.88200,0.72400,0.78900,0.75500,0.80600,0.82400,0.76700,plot,plot,plot
4,0.94500,nan,nan,0.75400,0.58700,0.68500,0.62400,0.66800,0.69000,0.68300,plot,plot,plot
5,0.98700,nan,nan,0.81100,0.73300,0.73000,0.70200,0.78100,0.70900,0.73500,plot,plot,plot
6,0.98300,nan,nan,0.64800,0.65700,0.63200,0.62600,0.63200,0.66500,0.63300,plot,plot,plot
7,0.98700,nan,nan,0.72600,0.69500,0.71400,0.69600,0.68400,0.70600,0.69700,plot,plot,plot
8,0.98700,nan,nan,0.76100,0.76200,0.75300,0.68500,0.70400,0.68400,0.71500,plot,plot,plot
9,0.96400,nan,nan,0.76000,0.76600,0.64400,0.64700,0.62600,0.63600,0.71600,plot,plot,plot


#### Violin Plots

In [43]:
#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/test_acc_g7_10000.png")




### Accuracy (unlabelled)

In [44]:
#rel_dn

os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/')

d_n='n36_gaussian_mixture_d7_10000'
sdir=dspec.loc[d_n].save_folder

#print('computing for dn: {0}'.format(d_n))
master_spec=pd.read_excel('combined_spec.xls',sheet_name=None)
dspec=master_spec['dataset_spec'] #write dataset spec shorthand
dspec.set_index("d_n",inplace=True) #set idx for easier

csi=master_spec['dataset_si'][d_n].values
candidate_si=csi[~np.isnan(csi)]
candidate_si=[int(c) for c in candidate_si]
cspec=dspec.loc[d_n] #current spec
model_accs={}
all_si=[s for s in candidate_si]
model_accs['s_i']=all_si




with NoStdStreams():
    for current_model in model_names_synthetic:
        vaccs=[]
        for s_i in candidate_si:
            try:
                vaccs.append(float(get_acc_ulab(dspec.save_folder.loc[d_n],s_i,current_model)))
            except:
                print('warning error for s_i: {0} and model: {1}'.format(s_i,current_model))
                vaccs.append(np.mean(vaccs))
        model_accs[current_model]=vaccs
    ma=pd.DataFrame(model_accs)

    for cm in model_names_synthetic:
        ma[cm]=ma[cm].astype(str)

    list_of_plots=[]
    list_of_synthetic_plots=[]
    list_of_synthetic_gumbel_plots=[]

    #get remaining s_i
    remaining_si=[i for i in ma.index]

    for s_i in remaining_si:
        im_fn=f'data_for_train_d_n_{d_n}_s_i_{s_i}_plot.png'
        fpath='../'+sdir+'/'+im_fn
        list_of_plots.append(fpath)
        im_fn=f'../{sdir}/synthetic_data/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'
        list_of_synthetic_plots.append(im_fn)
        im_fn=f'../{sdir}/synthetic_data_gumbel/synthetic_data_d_n_{d_n}_s_i_{s_i}_plot.png'

        list_of_synthetic_gumbel_plots.append(im_fn)
        
    ma['data_plot']=list_of_plots
    ma['synthetic_cgan_data']=list_of_synthetic_plots
    ma['synthetic_gumbel_cgan_data']=list_of_synthetic_gumbel_plots
    
    
    #all paths for decision boundary plots
    dict_of_paths={m:[get_soft_db_path(m,d_n,s_i,sdir) for s_i in remaining_si] for m in model_names_synthetic }
    

    style_dict={}
    style_dict={m:make_float for m in model_names_synthetic}
    style_dict.update({'data_plot':make_clickable,
                      'synthetic_cgan_data':make_clickable,
                      'synthetic_gumbel_cgan_data':make_clickable})
os.chdir('/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results')  
    
#ma.drop(columns=['s_i'],inplace=True)
#ma.style.format(style_dict)


#violin plots for model performamces
ma_i=copy.deepcopy(ma)

ma_i=ma_i.rename(columns={'CGAN_BASIC_DJ_SUPERVISED_CLASSIFIER':'CGAN_BASIC_SUPERVISED_CLASSIFIER',
                        'CGAN_GUMBEL_DJ_SUPERVISED_CLASSIFIER':'CGAN_GUMBEL_SUPERVISED_CLASSIFIER'})


ma_i=ma_i[model_names]
ma_i=ma_i.astype(float)

fig = go.Figure()
for m in model_names:
    fig.add_trace(go.Box(x=[m]*ma_i.shape[0],
                            y=ma_i[m],
                            name=m,
                        fillcolor=colours_dict[m],
                        marker_color="black"))


#fig.add_layout_image(dict(
#    source='causal_graphs/cg2.png',
#    x=0.7,
#    y=1,
#    )
#)

#ig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Accuracy')
fig.update_xaxes(showticklabels=False)

fig.update_layout(showlegend=False)

fig.show()
fig.write_image("/media/krillman/240GB_DATA/codes2/SSL_GCM/collating_results/images/ulab_acc_g7_10000.png",scale=5)



